# **신용카드 사용자 연체 예측 AI 경진대회**

1 RandomForest에 GridSearchCV로 파라미터 튜닝

2 XGBoost 모델에 BayesianOptimization로 파라미터 튜닝
<br><br>
참고

-데이콘 코드공유 : https://dacon.io/competitions/official/235713/codeshare/2476?page=1&dtype=recent

In [8]:
'''메인 라이브러리'''
import numpy as np
import pandas as pd
import os

'''시각화 관련 라이브러리'''
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl

%matplotlib inline

'''데이터 준비 관련 라이브러'''
from sklearn import preprocessing as pp 
from scipy.stats import pearsonr 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold 
from sklearn.metrics import log_loss 
from sklearn.metrics import precision_recall_curve, average_precision_score 
from sklearn.preprocessing import OneHotEncoder
 

'''알고리즘 관련 라이브러'''
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [3]:
from google.colab import drive
drive.mount._DEBUG = True
drive.mount('/content/drive', force_remount=True)

unset HISTFILE; export PS1="root@60dc80e44a8a-d50f72ba22714528b501b5ab1ba432fc: "
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/content# root@60dc80e44a8a-d50f72ba22714528b501b5ab1ba432fc: umount -f /content/drive || umount /content/drive; pkill -9 -x drive
umount: /content/drive: no mount point specified.
umount: /content/drive: no mount point specified.
root@60dc80e44a8a-d50f72ba22714528b501b5ab1ba432fc: pkill -9 -f /opt/google/drive/directoryprefetcher_binary
root@60dc80e44a8a-d50f72ba22714528b501b5ab1ba432fc: ( while `sleep 0.5`; do if [[ -d "/content/drive" && "$(ls -A /content/drive)" != "" ]]; then echo "google.colab.drive MOUNTED"; break; fi; done ) &
[1] 125
root@60dc80e44a8a-d50f72ba22714528b501b5ab1ba432fc: cat /tmp/tmps40yp8wr/drive.fifo | head -1 | ( /opt/google/drive/drive --features=fuse_max_background:1000,max_read_qps:1000,max_write_qps:1000,max_operation_batch_size:15,max_parallel_push_task_instances:1

In [37]:
os.chdir('./drive/MyDrive/dacon study/신용카드/')

FileNotFoundError: ignored

In [38]:
train = pd.read_csv('train.csv')
train = train.drop(['index'], axis=1)
train.fillna('NAN', inplace=True) 


test = pd.read_csv('test.csv')
test = test.drop(['index'], axis=1)
test.fillna('NAN', inplace=True)

submission=pd.read_csv('sample_submission.csv')#랜덤포레스트
submission2=pd.read_csv('sample_submission.csv')#xgboost

In [39]:
object_col = []
for col in train.columns:
    if train[col].dtype == 'object':
        object_col.append(col)

In [9]:
enc = OneHotEncoder()
enc.fit(train.loc[:,object_col])


train_onehot_df = pd.DataFrame(enc.transform(train.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
train.drop(object_col, axis=1, inplace=True)
train = pd.concat([train, train_onehot_df], axis=1)

In [10]:
test_onehot_df = pd.DataFrame(enc.transform(test.loc[:,object_col]).toarray(), 
             columns=enc.get_feature_names(object_col))
test.drop(object_col, axis=1, inplace=True)
test = pd.concat([test, test_onehot_df], axis=1)

모델링

In [11]:
train_x=train.drop('credit', axis=1)
train_y=train[['credit']]
test_x=test

In [12]:
X_train, X_val, y_train, y_val = train_test_split(train_x, train_y, 
                                                    stratify=train_y, test_size=0.25,
                                                    random_state = 10086)

In [16]:
from sklearn.model_selection import GridSearchCV
params = { 'n_estimators' : [100, 500],
           'max_depth' : [6, 8, 10],
           'min_samples_leaf' : [2,4,8],
           'min_samples_split' : [4,8,12],
           'max_features' : ['auto']
            }
model = RandomForestClassifier(random_state = 0, n_jobs = -1)
grid_cv = GridSearchCV(model, param_grid = params, cv = 3, n_jobs = -1)
grid_cv.fit(X_train, y_train)
print('최적 하이퍼 파라미터: ', grid_cv.best_params_)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


최적 하이퍼 파라미터:  {'max_depth': 10, 'max_features': 'auto', 'min_samples_leaf': 2, 'min_samples_split': 4, 'n_estimators': 100}


In [17]:
cv_model = RandomForestClassifier(n_estimators = 500, 
                                max_depth = 10,
                                max_features = 'auto',
                                min_samples_leaf = 2,
                                min_samples_split = 4,
                                random_state = 0,
                                n_jobs = -1)

In [18]:
from tensorflow.keras.utils import to_categorical

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=55)

sub=np.zeros((test_x.shape[0], 3))  
logloss_scores = []
model = cv_model
for n_fold, (train_index, val_index) in enumerate(folds.split(train_x, train_y)):
    X_train, X_val = train_x.iloc[train_index], train_x.iloc[val_index]
    y_train, y_val = train_y.iloc[train_index], train_y.iloc[val_index]
    model.fit(X_train, y_train)

    predictions=model.predict_proba(X_val)
    
    logloss=log_loss(to_categorical(y_val['credit']), predictions)
    logloss_scores.append(logloss)
    print(f"FOLD {n_fold} : logloss:{logloss}")
    sub+=model.predict_proba(test_x)
    
mean_outcome=np.mean(logloss_scores)
print("Mean:{}".format(mean_outcome))
my_submission=sub/folds.n_splits


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # This is added back by InteractiveShellApp.init_path()


FOLD 0 : logloss:0.796905409731399


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # This is added back by InteractiveShellApp.init_path()


FOLD 1 : logloss:0.7950316471543042


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # This is added back by InteractiveShellApp.init_path()


FOLD 2 : logloss:0.7954492070843666


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # This is added back by InteractiveShellApp.init_path()


FOLD 3 : logloss:0.7943526843791103


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # This is added back by InteractiveShellApp.init_path()


FOLD 4 : logloss:0.7939629747885696
Mean:0.79514038462755


In [19]:
submission.loc[:,1:]=my_submission

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  """Entry point for launching an IPython kernel.


In [20]:
submission

,index,0,1,2
0,26457,0.110083,0.189711,0.700206
1,26458,0.125512,0.183548,0.690940
2,26459,0.115596,0.200182,0.684222
3,26460,0.120987,0.170532,0.708482
4,26461,0.120304,0.182990,0.696706
...,...,...,...,...
9995,36452,0.121497,0.208575,0.669928
9996,36453,0.123939,0.229318,0.646743
9997,36454,0.091908,0.165504,0.742588
9998,36455,0.116550,0.182221,0.701229


In [36]:
submission.to_csv('RandomForest_GridSearchCV_submission.csv', index=False)

In [21]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp37-none-any.whl size=11687 sha256=8d2c3f09dd2907cfc33edf00ab9fc4f3b29c6e981bba0c89092cd48be2b449c3
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [22]:
from bayes_opt import BayesianOptimization

In [23]:
dtrain = xgb.DMatrix(X_train, y_train)

In [24]:
def xgb_evaluate(n_estimators, max_depth, subsample, colsample_bytree, learning_rate):
    params = {    
                'nthread':-1, 
                'learning rate':learning_rate,
                'max_depth': int(max_depth), 
                'n_estimators' : n_estimators,
                'subsample': subsample,
                'colsample_bytree': colsample_bytree, 
                'objective':'multi:softprob',
                'num_class':3,
                'eval_metric':'mlogloss',
                'seed':2018,
                'verbosity':1
             }

    cv_result = xgb.cv(params, dtrain, num_boost_round=50, nfold=5, early_stopping_rounds=50, seed=508)    

    # 최적화로 사용할 metric 반환
    # test-logloss-mean 값을 기준으로 마이너스 값이 출력 되므로 -1을 곱하여 return
    return -1.0 * cv_result['test-mlogloss-mean'].iloc[-1]

In [25]:
xgb_bo = BayesianOptimization(xgb_evaluate, 
                              {
                                'n_estimators' : (100,500),
                                'max_depth': (4, 10),
                                'subsample': (0.6, 1.0),
                                'colsample_bytree' :(0.6, 1.0),
                                'learning_rate': (0.01, 0.2)}
                              )

In [26]:
xgb_bo.maximize(init_points=3, n_iter = 10, acq='ei', xi=0.1, random_state=409)

|   iter    |  target   | colsam... | learni... | max_depth | n_esti... | subsample |
-------------------------------------------------------------------------------------
|  1        | -0.7732   |  0.8286   |  0.03641  |  5.108    |  493.9    |  0.7261   |
|  2        | -0.7741   |  0.7156   |  0.1747   |  5.292    |  210.6    |  0.7464   |
|  3        | -0.7714   |  0.8294   |  0.1165   |  5.431    |  220.4    |  0.6739   |
|  4        | -0.7723   |  0.6714   |  0.1317   |  5.821    |  343.3    |  0.8204   |
|  5        | -0.783    |  0.8974   |  0.1068   |  4.426    |  408.7    |  0.7417   |
|  6        | -0.7441   |  0.8274   |  0.02052  |  9.865    |  100.0    |  0.8487   |
|  7        | -0.7498   |  0.7023   |  0.1303   |  7.244    |  101.5    |  0.8489   |
|  8        | -0.7412   |  0.6307   |  0.08845  |  9.975    |  457.6    |  0.8066   |
|  9        | -0.7573   |  0.9966   |  0.08031  |  9.878    |  142.0    |  0.6351   |
|  10       | -0.7393   |  0.6201   |  0.0253   |  9.9

In [27]:
params = xgb_bo.max['params']
params['eval_metric'] = 'mlogloss'
params['objective'] = 'multi:softprob'
params['num_class'] = 3
params['max_depth'] = round(params['max_depth'])
params

{'colsample_bytree': 0.6200720440008622,
 'eval_metric': 'mlogloss',
 'learning_rate': 0.025298816713304244,
 'max_depth': 10,
 'n_estimators': 280.2948419275243,
 'num_class': 3,
 'objective': 'multi:softprob',
 'subsample': 0.9031134688939599}

In [28]:
model = xgb.cv(
    params=params,
    dtrain=dtrain,
    num_boost_round=1000,
    nfold=5,
    early_stopping_rounds=50,
    verbose_eval = 50
)

[0]	train-mlogloss:1.08699+0.000140309	test-mlogloss:1.0881+0.000204868
[50]	train-mlogloss:0.790243+0.00432326	test-mlogloss:0.84066+0.00590016
[100]	train-mlogloss:0.691858+0.00552703	test-mlogloss:0.784563+0.00825056
[150]	train-mlogloss:0.636707+0.00728794	test-mlogloss:0.764609+0.00926466
[200]	train-mlogloss:0.597157+0.00643381	test-mlogloss:0.753808+0.0102658
[250]	train-mlogloss:0.566186+0.00634954	test-mlogloss:0.747045+0.0106576
[300]	train-mlogloss:0.538678+0.00540828	test-mlogloss:0.742085+0.0114201
[350]	train-mlogloss:0.513411+0.00538342	test-mlogloss:0.737865+0.0115089
[400]	train-mlogloss:0.490518+0.00530283	test-mlogloss:0.734887+0.0114712
[450]	train-mlogloss:0.470214+0.00502366	test-mlogloss:0.732834+0.0117141
[500]	train-mlogloss:0.450983+0.00494096	test-mlogloss:0.731173+0.0116654
[550]	train-mlogloss:0.433044+0.00458541	test-mlogloss:0.730098+0.0119692
[600]	train-mlogloss:0.416926+0.0044116	test-mlogloss:0.729646+0.0121116
[650]	train-mlogloss:0.401263+0.0038256	

In [29]:
best_rounds = np.argmin(model['test-mlogloss-mean'])
print(best_rounds)

646


In [30]:
logloss_scores = []
for n_fold, (train_index, val_index) in enumerate(folds.split(train_x, train_y)):
    print(f'===================================={n_fold+1}============================================')
    X_train, X_val = train_x.iloc[train_index], train_x.iloc[val_index]
    y_train, y_val = train_y.iloc[train_index], train_y.iloc[val_index]

    d_train = xgb.DMatrix(X_train, y_train)
    d_val = xgb.DMatrix(data=X_val)
    

    xgb_model = xgb.train(params, d_train, num_boost_round=best_rounds)

    predictions=xgb_model.predict(d_val)
    
    logloss=log_loss(y_val, predictions)
    logloss_scores.append(logloss)
    print(f"FOLD {n_fold} : logloss:{logloss}")
    print(f'================================================================================\n\n')
        
mean_outcome=np.mean(logloss_scores)

print("Mean:{}".format(mean_outcome))

====================================1============================================
FOLD 0 : logloss:0.7190534332781208


====================================2============================================
FOLD 1 : logloss:0.7125755582022032


====================================3============================================
FOLD 2 : logloss:0.7145183553330937


====================================4============================================
FOLD 3 : logloss:0.7024229730962946


====================================5============================================
FOLD 4 : logloss:0.7093731681191409


Mean:0.7115886976057706


In [31]:
d_test = xgb.DMatrix(data=test_x)
pred_y = xgb_model.predict(d_test)
print(pred_y)

[[0.05161909 0.08977251 0.85860837]
 [0.2508247  0.16028404 0.58889127]
 [0.07112662 0.10877507 0.82009834]
 ...
 [0.01904378 0.11945664 0.8614996 ]
 [0.08802568 0.2527304  0.6592439 ]
 [0.0600154  0.20121184 0.73877275]]


In [32]:
submission2.loc[:,1:]=pred_y

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  """Entry point for launching an IPython kernel.


In [33]:
submission2

,index,0,1,2
0,26457,0.051619,0.089773,0.858608
1,26458,0.250825,0.160284,0.588891
2,26459,0.071127,0.108775,0.820098
3,26460,0.132598,0.122697,0.744705
4,26461,0.084599,0.150810,0.764592
...,...,...,...,...
9995,36452,0.174260,0.174349,0.651391
9996,36453,0.223710,0.314356,0.461934
9997,36454,0.019044,0.119457,0.861500
9998,36455,0.088026,0.252730,0.659244


In [35]:
submission2.to_csv('XGBoost_BayesianOptimization_submission.csv', index=False)